## <span style = "color:#FFD700"> Convolution Neural Network</span>

["스탠포드 cs231n : Lecture Collection | Convolutional Nerual Network"](https://www.youtube.com/playlist?list=PL3FW7Lu3i5JvHM8ljYj-zLfQRF3EO8sYv)

["스탠포드 cs231n : lecture note"](https://cs231n.github.io/convolutional-networks/#conv)

["위키독스:딥러닝 파이토치 교과서"](https://wikidocs.net/62306)

<br>
합성곱 신경망은, 커널이라는 작은 가중치 행렬을 fully-connected layer대신 사용하는 것이다.

특히 이미지 분류에 큰 이점을 보이는데, 왜냐면, 이 커널을 이미지 행렬에 곱해서 
특성맵이라는 결과 행렬을 만드는데,  이때, 작은 커널 행렬을 이미지에 겹쳐서 그 겹친 값들을 곱한 값을 특성맵으로 출력한다.

이런 식으로, layer를 구성하면, 이미지의 __공간적인 구조__ 를 살려서 레이어를 만드는 셈이라, 이미지와 관련된 작업을 처리하기에 유용한 모델이다.

(1) 합성곱의 정의와, 커널맵 (or Local Receptive Field )으로 합성곱층 (or feature map) 을 구성하는 방법

(2) 커널맵의 Stride, padding  설명

(3) 풀링층 만들기 ( Max pooling, Avg pooling )

(4) 케라스 API사용해서 CNN 구현하기
<br><br><br><br><br><br>

### <span style = "color:#1E90FF"> 합성곱의 정의, 정의를 커널맵을 통해 구현하기</span>

합성곱은 공학수학에서 배웠던 합성곱과 그 정의가 같다.

먼저, 10개의 원소로 이루어진 1차원 데이터 x를 생각해보자.

```python
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
```

그리고, 3개의 원소로 이루어진 커널맵 w를 생각하자.
```python
[2,4,5]
```

x,w의 합성곱은 아래와 같의 정의된다.

$$
y = x * w \rightarrow y[i] = \sum_{k=0}^{2}x[i+3-k]\,w[k]
$$

이 합성곱의 의미는, 커널맵을 슬라이싱 해가면서, 데이터 x와 겹치는 값에 대해 출력맵을 만드는 것이다.

```python
x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
w = [5, 4, 2]
```

$$
y[0] = \sum_{k=0}^{2}x[3-k]\,w[k] = 5 + 4 + 4 = 13
$$


```python
x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
w =    [5, 4, 2]
```

$$
y[1] = \sum_{k=0}^{2}x[4-k]\,w[k] = 5 + 8 + 6 = 19
$$

$$
\vdots \\
\vdots
$$

```python
x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
w =                      [5, 4, 2]
```

$$
y[7] = \sum_{k=0}^{2}x[10-k]\,w[k] = 35 + 32 + 18 = 85
$$

```python
y = [13, 19, ... , 85]                     
```
<br><br><br><br><br><br>

### <span style = "color:#1E90FF"> 3차원 이미지 텐서 [높이, 너비, 채널] 에 대한 합성곱 층 살펴보기</span>

위와 정확히 동일한 합성곱의 정의 및 연산방법으로, 3차원 이미지 텐서에 대해서 적용할 수 있다.
아래 사진을 참조하자.

![3차원 이미지 텐서 합성곱](../images/15_09_resized.png)

여기에서 입력 데이터는 3차원 이미지 텐서로, RGB 3개의 채널을 가지고 있다.

이제 m1xm2 사이즈의 3개의 depth를 가지고 있는 커널맵 5개를 준비하자.

하나의 커널맵은, y_1 x y_2 x 1 의 2차원 합성곱 층을 만든다. 그래서, 5개의 커널로 총 5개의 depth를 가지는 합성곱층이 형성된다.

하나의 커널맵이 3차원 이미지 텐서에 작용할때, 아래와 같이 커널맵의 3개의 채널각각이 이미지 텐서의 각 채널의 국소적인 부분(패치)에 작용하여, 하나의 값을 준다.

이 계산 과정은 아래 gif로 나와있고, 수식으로 표현하면 아래와 같다.
( 이미지텐서는 10 x 10 x 3 ,  커널맵은 3 x 3  x 3, stride = 1, padding = 0)

$$
\text{for each channel $c$, $Y_c$ is feature map}\\
Y(i,j)_c = W[:,:,c]*X[:,:,c] (i, j) = \sum_{k1}\sum_{k2}X[i-k1,j-k2]\,W[k1,k2]
$$

위에서 구한 $Y_c$ 는 이미지 텐서의 채널중 하나의 채널에 대해서 ( 즉, 10 x 10 x 1 텐서) 합성곱을 수행한 feature map이다.
이 $Y_c$ 텐서를 모두 더해서 최종적인 feature map $Z$ 를 만든다.

$$
\text{for each kernel $k$ from 1 to 5 , channel = $c$}\\
Z^{conv}_{k} = \sum_{channel=1}^{3}Y_{channel, k}\\
= \sum_{c=1}^{3}W[:,:,c]_k\,*\,X[:,:,c]_k
$$

여기서는 5개의 kernel중 하나의 커널에 대해서 만든 feature map $Z_k$ 를 구한 것이다. ( 사이즈는 [7 x 7 x 1] )

총 5개의 커널이 존재하므로, feature map tensor는 총 5층으로 이루어진 3차원 텐서가 된다. ( 사이즈 [7 x 7 x 5])

!["gif of evaluating convolution mapping of image tensor"](../images/imgs_6_CNN/conv_resized.gif)


위 그림에서 

입력 이미지 텐서 타입 : [7 x 7 x 3] <br>
커널 : [3 x 3 x 3] 2개<br>
Stride = 2

$$
{\footnotesize 출력 텐서 크기} = \frac{{\footnotesize 입력벡터 크기} - {\footnotesize 커널 크기} }{Stride}+1 = 3
$$

출력 텐서 : [3 x 3 x 2]

위에서 입력 이미지와 커널맵의 각 채널마다 합성곱을 진행하고,  결과인 출력맵에는 각 채널에서 진행한 합성곱들을 전부 더한 값이 나온다.

<br><br><br><br>


### <span style = "color:#1E90FF">  패딩, 풀링의 개념과, 스트라이드 설정으로 인한 데이터 손실</span>

패딩은 입력이미지 테두리에 0을 추가하는 역할을 한다.

![zero_padding](../images/imgs_6_CNN/zero_pad.png)

합성곱을 하면 스트라이드에 의해서 항상 출력맵의 크기가 줄어든다. (스트라이드가 클수록 크기는 더 줄어듦)

만약 입력의 크기가 [7 x 7] 이고, 커널맵의 크기가 [3 x 3],  stride = 1 일때,  출력맵은 [4 x 4] 이 된다.

이때 위 사진처럼 패딩을 하게 되면, 출력맵의 크기를 이전과 유지 할 수 있다.

패딩을 하는 이유는 다음과 같다.<br><br>
(1) 패딩을 하게 되면, 합성곱 출력의 사이즈를 유지할 수 있다.<br>
(2) 만약 출력맵의 크기가 줄면, 이는 데이터의 손실을 의미한다. ( 왜냐면, 특성맵의 정보들이 사진의 정보를 담고 있어서)


따라서, 만약 커널맵의 크기가 $m \in \{3,5\}$이라면, 패딩의 크기는 $\{1,2\}$ 로 설정하여 출력맵의 사이즈를 입력과 동일하게 유지 시킨다.

<br>


### <span style = "color:#1E90FF"> 풀링은 합성곱 레이어의 튀는 성질을 유지하면서 DownSampling하는것.</span>

풀링은 간단하게 말하면, 합성곱레이어의 크기를 줄이는 것이다. 

풀링은 2가지가 존재한다.

(1) Maxpooling : downsampling을 하지만, 사진의 눈에 띄는 값들 (즉, 경계선이나 명확한 구분선, 특징점) 을 살리고 싶을때 사용. (주로 maxpooling을 대부분 쓴다.)<br>
(2) AVGpooling : downsampling을 할때, 사진에의 평균적인 데이터/전체적인 경향성데이터를 보고 싶을때 (잘 안씀, 쓰면 성능이 올라가는 경우도 존재)

![pooling](../images/imgs_6_CNN/maxpooling.png)




풀링은 합성곱을 계산할때와 유사하게 maxpool 커널을 이용한다.

이때, maxpool커널의 크기가 [2 x 2] 라면, stride = 2 로 설정한다.

그러고, 사진을 sliding하면서 커널맵 안에 있는 값중 가장 큰 값을 선정해서 출력 맵에 저장한다.

이런 식으로 하면, 위 사진 처럼 2 x 2 칸으로 구획을 나누고, 가장 큰 값만 골라서 결국 레이어의 사이즈를 반갈죽 하는 형태가 된다.

##### <span style="color:#32CD32">풀링 레이어를 추가하는 이유?</span>

일단 CNN은 파라미터가 __너무 많다!!__ >> 모델이 Overfitting에 빠지기 매우 쉽다.

이때, 풀링 레이어를 추가하면 효과적으로 전체 파라미터의 수를 줄일 수 있다. 
( 맥스풀링에 의해서, 레이어의 특징을 살리면서 사이즈를 줄일 수 있다.)

### <span style = "color:#1E90FF"> 드디어, 케라스 API를 사용해서 CNN구현하기</span>